<div style="text-align: center; font-weight: bold;">
    <h1>Pipeline for Research ready EHR Datasets</h1>
    <h2>Part 3: Aggregating the EHR Data for Analysis</h2>
    <h4>Author: Vidul Ayakulangara Panickan</h3>
</div>




In [ ]:
## Step 7 Aggregating Data to patient level

The level of aggregation depends on your analysis needs. You can do a simple aggregation to 
get EHR code counts per patient as shown below

In [2]:
import pandas as pd
import os

os.makedirs("Aggregated_Data", exist_ok=True)

In [6]:
diagnoses = pd.read_csv("./Rolledup_Data/diagnoses_phecode.csv",dtype=str)
diagnoses

,subject_id,icd_code,icd_version,date,PheCode,Rollup_Status
0,10000032,5723,9,2180-05-06,571.81,1
1,10000032,78959,9,2180-05-06,572,1
2,10000032,5715,9,2180-05-06,571.51,1
3,10000032,07070,9,2180-05-06,070.3,1
4,10000032,496,9,2180-05-06,496,1
...,...,...,...,...,...,...
6356476,19999987,41401,9,2145-11-02,411.4,1
6356477,19999987,78039,9,2145-11-02,345.3,1
6356478,19999987,0413,9,2145-11-02,041,1
6356479,19999987,36846,9,2145-11-02,368.4,1


In [7]:
patient_phecode_counts=diagnoses.groupby(['subject_id', 'PheCode']).size().reset_index(name='count')
patient_phecode_counts



,subject_id,PheCode,count
0,10000032,070.3,4
1,10000032,071,3
2,10000032,260.1,1
3,10000032,276.12,3
4,10000032,276.13,2
...,...,...,...
3564825,19999987,368.4,1
3564826,19999987,411.4,1
3564827,19999987,430.2,1
3564828,19999987,585.1,1


In several cases, you might be interested in studying a particular condition say Asthma. First we identify phecodes of interest, here asthma has a phecode of "495". We will extract all patient data with PheCode 495

In [8]:
asthma_diagnoses = diagnoses[diagnoses['PheCode']=='495']
asthma_diagnoses

,subject_id,icd_code,icd_version,date,PheCode,Rollup_Status
724,10001725,49390,9,2110-04-11,495,1
878,10001884,J45909,10,2130-12-27,495,1
886,10001884,J45909,10,2130-12-06,495,1
955,10001884,J45909,10,2131-01-07,495,1
985,10001884,49390,9,2125-10-26,495,1
...,...,...,...,...,...,...
6355920,19999112,J45909,10,2116-10-28,495,1
6355931,19999112,J45909,10,2115-02-22,495,1
6355949,19999112,J45909,10,2115-02-11,495,1
6356068,19999442,49390,9,2146-12-19,495,1


You can perform a simple aggregation operation to get other diagnoses counts for these patients as done in previous cell.
Sometimes you will be interested in getting monthly counts around first diagnoses of asthma for the patient.

In [12]:
asthma_first_diagnoses = asthma_diagnoses.groupby('subject_id')['date'].min().reset_index()
asthma_first_diagnoses


,subject_id,date
0,10001725,2110-04-11
1,10001884,2125-10-26
2,10002800,2164-02-23
3,10003019,2174-09-16
4,10004296,2168-10-29
...,...,...
20311,19997760,2187-04-21
20312,19997887,2117-04-07
20313,19998350,2128-02-21
20314,19999112,2114-09-01


In [ ]:
We have 20316 unique patients with Astham diagnosis. These dates will be the index point which will be referenced to create monthly counts.